# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv = "Output File/cleanoutput.csv"

start_df = pd.read_csv(csv)

start_df


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,namatanai,PG,-3.67,152.43,78.44,80,10,5.39
1,pevek,RU,69.70,170.31,44.51,62,0,2.37
2,comodoro rivadavia,AR,-45.87,-67.50,44.60,87,75,3.36
3,buala,SB,-8.14,159.59,78.46,78,39,4.18
4,ushuaia,AR,-54.80,-68.30,30.20,92,75,6.93
...,...,...,...,...,...,...,...,...
560,necochea,AR,-38.55,-58.74,57.99,83,11,4.00
561,cacu,BR,-18.56,-51.13,82.90,27,0,9.35
562,auburn,US,47.31,-122.23,75.00,56,40,5.82
563,nabire,ID,-3.37,135.48,75.11,85,36,3.33


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
start_df['Humidity'] = start_df['Humidity'].astype(float)
locations = start_df[["Latitude", "Longitude"]]
hum = start_df["Humidity"].astype(float)


fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(locations, weights=hum,
                              dissipating=False, max_intensity=10,
                              point_radius=1)

fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_df = start_df

vacation_df1 = vacation_df[~(vacation_df['Wind Speed'] > 10)]
vacation_df2 = vacation_df1[~(vacation_df1['Cloudiness'] > 0)]
vacation_df3 = vacation_df2[~(vacation_df2['Max Temp'] > 85)]
vaca_final = vacation_df3[~(vacation_df3['Max Temp'] > 75)]
vaca_final.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
1,pevek,RU,69.70,170.31,44.51,62.0,0,2.37
18,hermanus,ZA,-34.42,19.23,55.99,88.0,0,5.28
40,saldanha,ZA,-33.01,17.94,62.60,67.0,0,5.82
49,bredasdorp,ZA,-34.53,20.04,57.20,87.0,0,4.70
68,hervey bay,AU,-25.30,152.85,55.99,91.0,0,7.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = vaca_final
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
1,pevek,RU,69.70,170.31,44.51,62.0,0,2.37
18,hermanus,ZA,-34.42,19.23,55.99,88.0,0,5.28
40,saldanha,ZA,-33.01,17.94,62.60,67.0,0,5.82
49,bredasdorp,ZA,-34.53,20.04,57.20,87.0,0,4.70
68,hervey bay,AU,-25.30,152.85,55.99,91.0,0,7.00


In [6]:
radious = 5000
tar_type = "lodging"

params = {
    "radius": radious,
    "type": tar_type,
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"]=f"{lat},{lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(url, params=params)
    hotel_res = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"]=hotel_res["results"][0]["name"]
    except(KeyError, IndexError):
        print("Skip")
#print(json.dumps(hotel_res,indent=4,sort_keys=True))

hotel_df.head()
    

C:\Users\bradb\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\bradb\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Skip
Skip
Skip
Skip
Skip
Skip
Skip


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
1,pevek,RU,69.70,170.31,44.51,62.0,0,2.37,Severnoye Zoloto
18,hermanus,ZA,-34.42,19.23,55.99,88.0,0,5.28,Misty Waves Boutique Hotel
40,saldanha,ZA,-33.01,17.94,62.60,67.0,0,5.82,Blue Bay Lodge
49,bredasdorp,ZA,-34.53,20.04,57.20,87.0,0,4.70,Bredasdorp Country Manor
68,hervey bay,AU,-25.30,152.85,55.99,91.0,0,7.00,Shelly Bay Resort


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))